In [1]:
"""
import SimpleITK as sitk
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import binary_fill_holes
from skimage.measure import label, regionprops
from skimage.morphology import binary_closing

def load_nii_image(file_path):
    nii_image = sitk.ReadImage(file_path)
    return sitk.GetArrayFromImage(nii_image), nii_image.GetSpacing()

def save_nii_image(image_array, spacing, file_path):
    nii_image = sitk.GetImageFromArray(image_array)
    nii_image.SetSpacing(spacing)
    sitk.WriteImage(nii_image, file_path)

def split_ribcage(ribcage_image, ribcage_spacing):
    left_ribcage = np.zeros_like(ribcage_image)
    right_ribcage = np.zeros_like(ribcage_image)
    mid_x = ribcage_image.shape[2] // 2
    left_ribcage[:, :, :mid_x] = ribcage_image[:, :, :mid_x]
    right_ribcage[:, :, mid_x:] = ribcage_image[:, :, mid_x:]
    return left_ribcage, right_ribcage

def process_single_rib(image_data, output_directory, affine):
    # 이미지 이진화
    threshold = 0.5
    binary_image = image_data > threshold

    closed_image = binary_closing(binary_image)

    # 3D 연결성 기반 클러스터링을 사용하여 뼈로 인식된 픽셀들을 그룹화
    labeled_image = label(closed_image)

    os.makedirs(output_directory, exist_ok=True)

    for region in regionprops(labeled_image):
        if region.area > 1000:
            single_rib_image = np.zeros_like(labeled_image)
            single_rib_image[labeled_image == region.label] = 1
            output_file_name = f'rib_{region.label}.nii.gz'
            output_path = os.path.join(output_directory, output_file_name)
            single_rib_nifti = nib.Nifti1Image(single_rib_image, affine)
            nib.save(single_rib_nifti, output_path)

if __name__ == "__main__":
    input_file_path = "C:/RIB/prediction_0_2000_v2/RibFrac541-image_pred.nii"

    ribcage_image, ribcage_spacing = load_nii_image(input_file_path)
    left_ribcage, right_ribcage = split_ribcage(ribcage_image, ribcage_spacing)

    left_output_file_path = "C:/RIB/code/code_final/Numbering/Ribfrac541/left_ribcage.nii"
    right_output_file_path = "C:/RIB/code/code_final/Numbering/Ribfrac541/right_ribcage.nii"

    save_nii_image(left_ribcage, ribcage_spacing, left_output_file_path)
    save_nii_image(right_ribcage, ribcage_spacing, right_output_file_path)

    # Left ribcage processing
    nifti_image = nib.load(left_output_file_path)
    image_data = nifti_image.get_fdata()

    output_directory = 'C:/RIB/code/code_final/Numbering/Ribfrac541/output_directory_left'
    process_single_rib(image_data, output_directory, nifti_image.affine)

    # Right ribcage processing
    nifti_image = nib.load(right_output_file_path)
    image_data = nifti_image.get_fdata()

    output_directory = 'C:/RIB/code/code_final/Numbering/Ribfrac541/output_directory_right'
    process_single_rib(image_data, output_directory, nifti_image.affine)
"""

In [1]:
#SimpleITK,Nibabel,Numpy,Scipy 그리고 Scikit-image 등의 Python 라이브러리를 이용하여 늑골 이미지를 처리하고 분석하는 작업을 수행한다.
import SimpleITK as sitk
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import binary_fill_holes
from skimage.measure import label, regionprops
from skimage.morphology import binary_closing

def load_nii_image(file_path): #Nifti 이미지를 불러와서 Numpy 배열로 변환하고 이미지 픽셀 간격(spacing)정보를 함께 반환
    nii_image = sitk.ReadImage(file_path)
    return sitk.GetArrayFromImage(nii_image), nii_image.GetSpacing()

def save_nii_image(image_array, spacing, file_path):# 주어진 Numpy 배열과 픽셀 간격 정보를 이용하여 Nifti 이미지를 생성하고 주어진 파일 경로에 저장
    nii_image = sitk.GetImageFromArray(image_array)
    nii_image.SetSpacing(spacing)
    sitk.WriteImage(nii_image, file_path)

def split_ribcage(ribcage_image, ribcage_spacing): #주어진 늑골 이미지를 좌우로 분할하여 왼쪽 늑골 이미지와 오른쪽 늑골 이미지를 생성
    left_ribcage = np.zeros_like(ribcage_image)
    right_ribcage = np.zeros_like(ribcage_image)
    mid_x = ribcage_image.shape[2] // 2
    left_ribcage[:, :, :mid_x] = ribcage_image[:, :, :mid_x]
    right_ribcage[:, :, mid_x:] = ribcage_image[:, :, mid_x:]
    return left_ribcage, right_ribcage

def process_single_rib(image_data, output_directory, affine): #주어진 늑골 이미지를 이용하여 이진 이미지를 생성하고 이를 닫힘 연산을 통해 스무딩한 후에 라벨링을 진행 그리고 
                                                              #각 라벨에 해당하는 영역의 크기가 1000보다 큰 경우 해당 영역을 분리하여 별도의 늑골 이미지를 생성하고 이를 NIFTI 파일로 저장
    threshold = 0.5
    binary_image = image_data > threshold

    closed_image = binary_closing(binary_image)
    labeled_image = label(closed_image)

    os.makedirs(output_directory, exist_ok=True)

    for region in regionprops(labeled_image):
        if region.area > 1000:
            single_rib_image = np.zeros_like(labeled_image)
            single_rib_image[labeled_image == region.label] = 1
            output_file_name = f'rib_{region.label}.nii.gz'
            output_path = os.path.join(output_directory, output_file_name)
            single_rib_nifti = nib.Nifti1Image(single_rib_image, affine)
            nib.save(single_rib_nifti, output_path)

if __name__ == "__main__":
    base_input_directory = "C:/RIB/prediction_0_2000_v2"
    base_output_directory = "C:/RIB/code/code_final/Numbering"

    for ribfrac_number in range(645, 672):
        input_file_path = f"{base_input_directory}/RibFrac{ribfrac_number}-image_pred.nii"
        if os.path.exists(input_file_path):
            ribcage_image, ribcage_spacing = load_nii_image(input_file_path)
            left_ribcage, right_ribcage = split_ribcage(ribcage_image, ribcage_spacing)

            ribfrac_directory = f"{base_output_directory}/Ribfrac{ribfrac_number}"
            os.makedirs(ribfrac_directory, exist_ok=True)

            left_output_file_path = f"{ribfrac_directory}/left_ribcage.nii"
            right_output_file_path = f"{ribfrac_directory}/right_ribcage.nii"

            #save_nii_image(left_ribcage, ribcage_spacing, left_output_file_path)
            #save_nii_image(right_ribcage, ribcage_spacing, right_output_file_path)

            nifti_image = nib.load(left_output_file_path)
            image_data = nifti_image.get_fdata()

            output_directory = f'{ribfrac_directory}/output_directory_left'
            process_single_rib(image_data, output_directory, nifti_image.affine)

            nifti_image = nib.load(right_output_file_path)
            image_data = nifti_image.get_fdata()

            output_directory = f'{ribfrac_directory}/output_directory_right'
            process_single_rib(image_data, output_directory, nifti_image.affine)


In [ ]:
"""
import os
import cv2
import numpy as np
import nibabel as nib
from skimage.measure import label, regionprops

# Define the function to process files
def process_files(files, directory, side):
    for file in files:
        file_path = os.path.join(directory, file)
        nifti_image = nib.load(file_path)
        image_data = nifti_image.get_fdata()

        labeled_image = label(image_data)
        for region in regionprops(labeled_image):
            if region.area > 500: 
                bones[file] = {"side": side, "coords": region.coords, "z_coordinates": set(int(coord[2]) for coord in region.coords)}

# Loop through all directories from Ribfrac541 to Ribfrac670
for i in range(540, 671):
    left_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/output_directory_left'
    right_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/output_directory_right'

    # Check if directories exist
    if not os.path.isdir(left_directory) or not os.path.isdir(right_directory):
        print(f"Skipping Ribfrac{i} due to missing directory")
        continue

    print(f"Processing Ribfrac{i}")

    left_files = [file for file in os.listdir(left_directory) if file.endswith('.nii.gz')]
    right_files = [file for file in os.listdir(right_directory) if file.endswith('.nii.gz')]

    merged_data = None

    for files, directory in [(left_files, left_directory), (right_files, right_directory)]:
        for file in files:
            file_path = os.path.join(directory, file)
            nifti_image = nib.load(file_path)
            image_data = nifti_image.get_fdata()

            if merged_data is None:
                merged_data = np.zeros_like(image_data)

            merged_data = np.maximum(merged_data, image_data)

    bones = {}

    process_files(left_files, left_directory, 'R') 
    process_files(right_files, right_directory, 'L')

    sorted_bones = sorted(bones.items(), key=lambda x: (x[1]['side'], min(x[1]['coords'], key=lambda coord: coord[0])[2]), reverse=True)

    for side in ['R', 'L']:  
        start = 1
        for file, bone_info in sorted_bones:
            if bone_info['side'] == side:
                bone_number = f"{side}{start}"
                print(f"{file} is assigned with {bone_info['side']} number {bone_number}.")
                bone_info['number'] = bone_number
                start += 1

    output_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/number_06_15'
    os.makedirs(output_directory, exist_ok=True)

    for z in range(merged_data.shape[2] - 1, -1, -1):
        slice_data = merged_data[:, :, z]

        normalized_data = ((slice_data / np.max(slice_data)) * 255).astype(np.uint8)

        numbers_to_render = []
        for file, bone_info in bones.items():
            if z in bone_info['z_coordinates']:
                coords_on_slice = [coord for coord in bone_info['coords'] if coord[2] == z]
                top_coord = min(coords_on_slice, key=lambda coord: coord[0])
                numbers_to_render.append((bone_info['number'], top_coord))

        bgr_data = cv2.cvtColor(normalized_data, cv2.COLOR_GRAY2BGR)
        for number, coord in numbers_to_render:
            cv2.putText(bgr_data, number, (int(coord[1]), int(coord[0])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        output_file_path = os.path.join(output_directory, f"slice_{merged_data.shape[2]-z:03d}.png")
        cv2.imwrite(output_file_path, bgr_data)

print("The images have been sliced and saved.")
"""

In [ ]:
#오른쪽 부분만-왼쪽
import os
import cv2
import numpy as np
import nibabel as nib
from skimage.measure import label, regionprops

# Define the function to process files
def process_files(files, directory, side):
    for file in files:
        file_path = os.path.join(directory, file)
        nifti_image = nib.load(file_path)
        image_data = nifti_image.get_fdata()

        labeled_image = label(image_data)
        for region in regionprops(labeled_image):
            if region.area > 500: 
                bones[file] = {"side": side, "coords": region.coords, "z_coordinates": set(int(coord[2]) for coord in region.coords)}

# Loop through all directories from Ribfrac541 to Ribfrac670
for i in range(540, 670):
    right_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/output_directory_right'

    # Check if directories exist
    if not os.path.isdir(right_directory):
        print(f"Skipping Ribfrac{i} due to missing directory")
        continue

    print(f"Processing Ribfrac{i}")

    right_files = [file for file in os.listdir(right_directory) if file.endswith('.nii.gz')]

    merged_data = None

    for file in right_files:
        file_path = os.path.join(right_directory, file)
        nifti_image = nib.load(file_path)
        image_data = nifti_image.get_fdata()

        if merged_data is None:
            merged_data = np.zeros_like(image_data)

        merged_data = np.maximum(merged_data, image_data)

    bones = {}

    process_files(right_files, right_directory, 'R')

    sorted_bones = sorted(bones.items(), key=lambda x: (x[1]['side'], min(x[1]['coords'], key=lambda coord: coord[0])[2]), reverse=True)

    start = 1
    for file, bone_info in sorted_bones:
        if bone_info['side'] == 'R':
            bone_number = f"L{start}"
            print(f"{file} is assigned with number {bone_number}.")
            bone_info['number'] = bone_number
            start += 1

    output_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/number_left'
    os.makedirs(output_directory, exist_ok=True)

    for z in range(merged_data.shape[2] - 1, -1, -1): #3D 이미지를 z축을 따라 슬라이스하는 데 사용- range(...)를 통해 z축의 맨 위 부터 맨 아래까지 역순으로 슬라이스
        slice_data = merged_data[:, :, z] #현재 z 위치에서의 2d 슬라이스를 가져옵니다.

        normalized_data = ((slice_data / np.max(slice_data)) * 255).astype(np.uint8) #슬라이스 데이터를 정규화하고, 8비트 부호없는 정수형으로 변환한다. 이를 통해 0부터 255 사이의 값으로 픽셀 강도를 표현

        numbers_to_render = [] #현재 슬라이스에서 늑골 번호를 그릴 위치를  결정. 각 늑골에 대해 그 늑골이 현재 슬라이스에 있는지 확인하고 있으면 그 늑골의 최상단 좌표를 구해 numbers_to_render에 추가
        for file, bone_info in bones.items():
            if z in bone_info['z_coordinates']:
                coords_on_slice = [coord for coord in bone_info['coords'] if coord[2] == z]
                top_coord = min(coords_on_slice, key=lambda coord: coord[0])
                numbers_to_render.append((bone_info['number'], top_coord))

        bgr_data = cv2.cvtColor(normalized_data, cv2.COLOR_GRAY2BGR) #흑백 이미지를 BGR 이미지로 변환. 이를 통해 늑골 번호를 색상으로 표시
        for number, coord in numbers_to_render: #numbers_to_render에 저장된 각 늑골 번호와 좌표에 대해 그 번호를 해당 좌표 위치에 그린다.
            cv2.putText(bgr_data, number, (int(coord[1]), int(coord[0])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        output_file_path = os.path.join(output_directory, f"slice_{merged_data.shape[2]-z:03d}.png") #번호가 그려진 슬라이스 이미지를 파일로 저장
        cv2.imwrite(output_file_path, bgr_data)

print("The images have been sliced and saved.")


In [ ]:
#왼쪽부분만-오른쪽
import os
import cv2
import numpy as np
import nibabel as nib
from skimage.measure import label, regionprops

# Define the function to process files
def process_files(files, directory, side):
    for file in files:
        file_path = os.path.join(directory, file)
        nifti_image = nib.load(file_path)
        image_data = nifti_image.get_fdata()

        labeled_image = label(image_data)
        for region in regionprops(labeled_image):
            if region.area > 500: 
                bones[file] = {"side": side, "coords": region.coords, "z_coordinates": set(int(coord[2]) for coord in region.coords)}

# Loop through all directories from Ribfrac541 to Ribfrac670
for i in range(540, 670):
    left_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/output_directory_left'

    # Check if directories exist
    if not os.path.isdir(left_directory):
        print(f"Skipping Ribfrac{i} due to missing directory")
        continue

    print(f"Processing Ribfrac{i}")

    left_files = [file for file in os.listdir(left_directory) if file.endswith('.nii.gz')]

    merged_data = None

    for file in left_files:
        file_path = os.path.join(left_directory, file)
        nifti_image = nib.load(file_path)
        image_data = nifti_image.get_fdata()

        if merged_data is None:
            merged_data = np.zeros_like(image_data)

        merged_data = np.maximum(merged_data, image_data)

    bones = {}

    process_files(left_files, left_directory, 'L')

    sorted_bones = sorted(bones.items(), key=lambda x: (x[1]['side'], min(x[1]['coords'], key=lambda coord: coord[0])[2]), reverse=True)

    start = 1
    for file, bone_info in sorted_bones:
        if bone_info['side'] == 'L':
            bone_number = f"R{start}"
            print(f"{file} is assigned with number {bone_number}.")
            bone_info['number'] = bone_number
            start += 1

    output_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/number_right'
    os.makedirs(output_directory, exist_ok=True)

    for z in range(merged_data.shape[2] - 1, -1, -1):
        slice_data = merged_data[:, :, z]

        normalized_data = ((slice_data / np.max(slice_data)) * 255).astype(np.uint8)

        numbers_to_render = []
        for file, bone_info in bones.items():
            if z in bone_info['z_coordinates']:
                coords_on_slice = [coord for coord in bone_info['coords'] if coord[2] == z]
                top_coord = min(coords_on_slice, key=lambda coord: coord[0])
                numbers_to_render.append((bone_info['number'], top_coord))

        bgr_data = cv2.cvtColor(normalized_data, cv2.COLOR_GRAY2BGR)
        for number, coord in numbers_to_render:
            cv2.putText(bgr_data, number, (int(coord[1]), int(coord[0])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        output_file_path = os.path.join(output_directory, f"slice_{merged_data.shape[2]-z:03d}.png")
        cv2.imwrite(output_file_path, bgr_data)

print("The images have been sliced and saved.")


In [19]:
import cv2
import numpy as np

# Loop through all directories from Ribfrac541 to Ribfrac670
for i in range(540, 670):

    # Define the directories where the left and right images are stored
    left_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/number_left'
    right_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/number_right'

    # Check if directories exist
    if not os.path.isdir(left_directory) or not os.path.isdir(right_directory):
        print(f"Skipping Ribfrac{i} due to missing directory")
        continue

    print(f"Processing Ribfrac{i}")

    # Iterate through all slices
    for z in range(999):  # Assuming max number of slices is 999
        left_image_path = os.path.join(left_directory, f"slice_{z:03d}.png")
        right_image_path = os.path.join(right_directory, f"slice_{z:03d}.png")

        # Check if both images exist
        if not os.path.isfile(left_image_path) or not os.path.isfile(right_image_path):
            print(f"Skipping slice {z} due to missing image")
            continue

        # Load images
        left_image = cv2.imread(left_image_path)
        right_image = cv2.imread(right_image_path)

        # Merge images. Note: This assumes that the images are properly aligned!
        merged_image = cv2.addWeighted(left_image, 0.5, right_image, 0.5, 0)

        # Save the merged image
        output_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/number_merged'
        os.makedirs(output_directory, exist_ok=True)
        output_image_path = os.path.join(output_directory, f"slice_{z:03d}.png")
        cv2.imwrite(output_image_path, merged_image)

print("The images have been merged and saved.")


Skipping Ribfrac540 due to missing directory
Processing Ribfrac541
Skipping slice 0 due to missing image
Skipping slice 365 due to missing image
Skipping slice 366 due to missing image
Skipping slice 367 due to missing image
Skipping slice 368 due to missing image
Skipping slice 369 due to missing image
Skipping slice 370 due to missing image
Skipping slice 371 due to missing image
Skipping slice 372 due to missing image
Skipping slice 373 due to missing image
Skipping slice 374 due to missing image
Skipping slice 375 due to missing image
Skipping slice 376 due to missing image
Skipping slice 377 due to missing image
Skipping slice 378 due to missing image
Skipping slice 379 due to missing image
Skipping slice 380 due to missing image
Skipping slice 381 due to missing image
Skipping slice 382 due to missing image
Skipping slice 383 due to missing image
Skipping slice 384 due to missing image
Skipping slice 385 due to missing image
Skipping slice 386 due to missing image
Skipping slice 

KeyboardInterrupt: 

In [ ]:
#뼈, 텍스트 밝게
import cv2
import numpy as np
import os

# Function to increase brightness for certain areas
def increase_brightness_mask(image, mask, value=50):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[mask > lim] += value

    final_hsv = cv2.merge((h, s, v))
    image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return image

# Loop through all directories from Ribfrac541 to Ribfrac670
for i in range(540, 670):
    # Define the directories where the left and right images are stored
    left_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/number_left'
    right_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/number_right'

    # Check if directories exist
    if not os.path.isdir(left_directory) or not os.path.isdir(right_directory):
        print(f"Skipping Ribfrac{i} due to missing directory")
        continue

    print(f"Processing Ribfrac{i}")

    # Iterate through all slices
    for z in range(999):  # Assuming max number of slices is 999
        left_image_path = os.path.join(left_directory, f"slice_{z:03d}.png")
        right_image_path = os.path.join(right_directory, f"slice_{z:03d}.png")

        # Check if both images exist
        if not os.path.isfile(left_image_path) or not os.path.isfile(right_image_path):
            print(f"Skipping slice {z} due to missing image")
            continue

        # Load images
        left_image = cv2.imread(left_image_path)
        right_image = cv2.imread(right_image_path)

        # Merge images
        merged_image = cv2.addWeighted(left_image, 0.5, right_image, 0.5, 0)

        # Create a mask of the bone and text areas (assuming they are darker)
        grayscale_image = cv2.cvtColor(merged_image, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(grayscale_image, 128, 255, cv2.THRESH_BINARY_INV)

        # Increase brightness of the bone and text areas
        brighter_merged_image = increase_brightness_mask(merged_image, mask)

        # Save the merged image
        output_directory = f'C:/RIB/code/code_final/Numbering/Ribfrac{i}/number_merged'
        os.makedirs(output_directory, exist_ok=True)
        output_image_path = os.path.join(output_directory, f"slice_{z:03d}.png")
        cv2.imwrite(output_image_path, brighter_merged_image)

print("The images have been merged and saved.")
